In [2]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
import numpy as np 
import pickle 
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("Churn_Modelling.csv")

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [6]:
gender_encoder = LabelEncoder()
data["Gender"] = gender_encoder.fit_transform(data["Gender"])

In [7]:
ohe = OneHotEncoder()
geo_encoder = ohe.fit_transform(data[["Geography"]]).toarray()

In [8]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=ohe.get_feature_names_out(['Geography']))


In [9]:
data.drop(["Geography"],axis=1,inplace=True)

In [10]:
final_df = pd.concat([data,geo_encoded_df],axis=1)

In [11]:
final_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [12]:
X = final_df.drop("Exited",axis=1)
y = final_df["Exited"]
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [14]:
with open("label_encoder_gender.pkl","wb") as leg:
    pickle.dump(gender_encoder,leg)

with open("onehotEncoder.pkl","wb") as ohf:
    pickle.dump(ohe,ohf)
    
with open("scaler.pkl","wb") as sr:
    pickle.dump(scaler,sr)

In [15]:
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import TensorBoard,EarlyStopping

In [16]:
model = Sequential([
    Dense(units=64,activation="relu",input_shape=(X_train.shape[1],)),
    Dense(units=32,activation="relu"),
    Dense(1,activation="sigmoid")
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")

In [19]:
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
es = EarlyStopping(patience=10,monitor="val_loss",restore_best_weights=True)
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [20]:
model.fit(X_train,y_train,epochs=50,callbacks=[es,tensorflow_callback],validation_data=(X_test,y_test))

Epoch 1/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4524 - accuracy: 0.8019 - val_loss: 0.3921 - val_accuracy: 0.8420
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3860 - accuracy: 0.8429 - val_loss: 0.3612 - val_accuracy: 0.8510
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3565 - accuracy: 0.8549 - val_loss: 0.3489 - val_accuracy: 0.8580
Epoch 4/50
250/250 [==============================] - 0s 985us/step - loss: 0.3445 - accuracy: 0.8589 - val_loss: 0.3463 - val_accuracy: 0.8575
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3385 - accuracy: 0.8624 - val_loss: 0.3430 - val_accuracy: 0.8560
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3328 - accuracy: 0.8635 - val_loss: 0.3542 - val_accuracy: 0.8565
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3305 - accuracy: 0.8651 - val_loss: 0.3385 - val_accuracy: 0.8605
Epoc

In [21]:
model.save("model.h5")

In [22]:
%load_ext tensorboard 

In [23]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2580), started 0:19:58 ago. (Use '!kill 2580' to kill it.)